In [1]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.6 MB/s eta 0:00:00


In [2]:
!pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 57.1 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer, util
import fitz  # PyMuPDF for PDFs
import numpy as np
import re
import requests

# Initialize Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# NVIDIA API details
NVIDIA_API_URL = "https://integrate.api.nvidia.com/v1/chat/completions"
API_KEY = "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"  # Replace with your valid NVIDIA API key

# Function to load and parse Q&A from PDF
def extract_qa_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = " ".join([page.get_text() for page in doc])

    # Extract structured Q&A pairs using regex
    qa_pairs = []
    qa_pattern = r'\"question\":\s*\"(.*?)\",\s*\"answer\":\s*\"(.*?)\"'
    matches = re.findall(qa_pattern, text, re.DOTALL)

    for match in matches:
        question, answer = match
        qa_pairs.append({"question": question.strip(), "answer": answer.strip()})

    return qa_pairs

# Retrieve best-matching answer and expand it contextually
def get_best_answer(user_question, qa_pairs):
    questions = [qa["question"] for qa in qa_pairs]
    question_embeddings = model.encode(questions, convert_to_tensor=True)
    user_embedding = model.encode(user_question, convert_to_tensor=True)

    # Compute similarity scores
    scores = util.cos_sim(user_embedding, question_embeddings).cpu().numpy()
    best_match_idx = np.argmax(scores)
    best_match_score = scores[0][best_match_idx]

    # Set a similarity threshold to determine if it's a valid match
    if best_match_score > 0.5:
        return qa_pairs[best_match_idx]["answer"]
    else:
        return None  # No exact match found

# Generate elaborated answer by providing the original answer as context with logical reasoning
def generate_elaborated_answer(question, short_answer):
    headers = {"Authorization": f"Bearer {API_KEY}"}

    # Modify the prompt to provide a detailed, case study-based response with logical reasoning
    prompt = f"""
    The question below involves a case study or logical reasoning. Provide a detailed and thoughtful response that addresses the question comprehensively, including real-world examples or theoretical scenarios. The answer should explain the reasoning behind the conclusion and end with a clear, actionable insight. Do not mention stages or steps explicitly.

    Question: {question}
    Answer: {short_answer}
    """

    payload = {
        "model": "microsoft/phi-3-mini-128k-instruct",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.2,
        "top_p": 0.9,
        "max_tokens": 1500,  # Increased token limit for longer responses
    }

    response = requests.post(NVIDIA_API_URL, json=payload, headers=headers)
    response_data = response.json()

    if response.status_code == 200 and "choices" in response_data:
        return response_data["choices"][0]["message"]["content"].strip()
    else:
        return "I'm unable to find an answer. Please try again later with a different question."

# Main function with a limit of 5 questions
def main(pdf_path):
    qa_pairs = extract_qa_from_pdf(pdf_path)
    question_count = 0  # Counter for questions

    while question_count < 5:  # Limit to 5 questions
        user_question = input(f"({question_count+1}/5) Enter your question (or type 'exit' to quit): ")
        if user_question.lower() == "exit":
            break

        answer = get_best_answer(user_question, qa_pairs)

        if answer:
            # Generate a detailed and expanded version of the answer with case study and logical reasoning
            elaborated_answer = generate_elaborated_answer(user_question, answer)

            # Ensure the response ends properly
            if elaborated_answer and elaborated_answer[-1] not in ['.', '!', '?']:
                elaborated_answer += "."

            print(f"\n✅ Elaborated Answer: {elaborated_answer}\n")
        else:
            print("\n❌ Exact match not found. Generating a detailed response...")

            # No fallback generation since it's already handled by elaboration
            print(f"\n🤖 No answer available for this question.\n")

        question_count += 1  # Increment question count

    print("\n🎯 You have reached the limit of 5 questions. Exiting...")

pdf_path = "/content/Campus_Pal Content.pdf"
main(pdf_path)

(1/5) Enter your question (or type 'exit' to quit): If a student skilled in Python, Java, and data analysis is looking to  specialize, which programs at Amrita Vishwa Vidyapeetham would you  recommend, and why? 

✅ Elaborated Answer: Given the student's proficiency in Python, Java, and data analysis, I would recommend the following programs at Amrita Vishwa Vidyapeetham:

1. Master of Science in Data Science: This program is designed to provide students with a strong foundation in data science, including data mining, machine learning, and big data analytics. The student's existing skills in Python and data analysis would be a significant advantage in this program, as Python is a widely used language in data science, and the program covers various data analysis techniques.

2. Master of Science in Artificial Intelligence: This program focuses on the development of intelligent systems and algorithms, including machine learning, deep learning, and natural language processing. The student'

In [6]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [10]:
import gradio as gr
from sentence_transformers import SentenceTransformer, util
import fitz  # PyMuPDF for PDFs
import numpy as np
import re
import requests

# Initialize Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# NVIDIA API details
NVIDIA_API_URL = "https://integrate.api.nvidia.com/v1/chat/completions"
API_KEY = "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"  # Replace with your valid NVIDIA API key

# Function to load and parse Q&A from PDF
def extract_qa_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = " ".join([page.get_text() for page in doc])

    # Extract structured Q&A pairs using regex
    qa_pairs = []
    qa_pattern = r'\"question\":\s*\"(.*?)\",\s*\"answer\":\s*\"(.*?)\"'
    matches = re.findall(qa_pattern, text, re.DOTALL)

    for match in matches:
        question, answer = match
        qa_pairs.append({"question": question.strip(), "answer": answer.strip()})

    return qa_pairs

# Retrieve best-matching answer and expand it contextually
def get_best_answer(user_question, qa_pairs):
    questions = [qa["question"] for qa in qa_pairs]
    question_embeddings = model.encode(questions, convert_to_tensor=True)
    user_embedding = model.encode(user_question, convert_to_tensor=True)

    # Compute similarity scores
    scores = util.cos_sim(user_embedding, question_embeddings).cpu().numpy()
    best_match_idx = np.argmax(scores)
    best_match_score = scores[0][best_match_idx]

    # Set a similarity threshold to determine if it's a valid match
    if best_match_score > 0.5:
        return qa_pairs[best_match_idx]["answer"]
    else:
        return None  # No exact match found

# Generate elaborated answer by providing the original answer as context with logical reasoning
def generate_elaborated_answer(question, short_answer):
    headers = {"Authorization": f"Bearer {API_KEY}"}

    # Modify the prompt to provide a detailed, case study-based response with logical reasoning
    prompt = f"""
    The question below involves a case study or logical reasoning. Provide a detailed and thoughtful response that addresses the question comprehensively, including real-world examples or theoretical scenarios. The answer should explain the reasoning behind the conclusion and end with a clear, actionable insight. Do not mention stages or steps explicitly.

    Question: {question}
    Answer: {short_answer}
    """

    payload = {
        "model": "microsoft/phi-3-mini-128k-instruct",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.2,
        "top_p": 0.9,
        "max_tokens": 1500,  # Increased token limit for longer responses
    }

    response = requests.post(NVIDIA_API_URL, json=payload, headers=headers)
    response_data = response.json()

    if response.status_code == 200 and "choices" in response_data:
        return response_data["choices"][0]["message"]["content"].strip()
    else:
        return "I'm unable to find an answer. Please try again later with a different question."

# Gradio interface
def qa_interface(pdf, question):
    qa_pairs = extract_qa_from_pdf(pdf.name)
    answer = get_best_answer(question, qa_pairs)

    if answer:
        elaborated_answer = generate_elaborated_answer(question, answer)
        if elaborated_answer and elaborated_answer[-1] not in ['.', '!', '?']:
            elaborated_answer += "."
        return f"✅ Elaborated Answer: {elaborated_answer}"
    else:
        ai_answer = generate_elaborated_answer(question, "No exact match found")
        return f"🤖 AI-Generated Answer: {ai_answer}"

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("## Amrita College: Your Interactive Guide to Campus Insights")
    pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
    question_input = gr.Textbox(label="Enter your question", placeholder="Type your question here...")
    answer_output = gr.Textbox(label="Answer", lines=5, interactive=False)

    submit_button = gr.Button("Submit")

    submit_button.click(qa_interface, inputs=[pdf_input, question_input], outputs=[answer_output])

# Launch the Gradio app
demo.launch(share=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://be993c51f010b01638.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
